Example of predicted answer format

In [ ]:
# {
#     "1": {
#         "Question": "What is the population of Central Greece?",
#         "Query": "SELECT DISTINCT ?population WHERE { <http://yago-knowledge.org/resource/Central_Greece_(region)> y2geoo:hasGAG_Population ?population }",
#         "Annotation": {
#             "pred_instances": ["<http://yago-knowledge.org/resource/Central_Greece_(region)>" , "yago:Greece"],
#             "pred_instances_disambiguated": ["<http://yago-knowledge.org/resource/Central_Greece_(region)>"],
#             "pred_properties": ["y2geoo:hasGAG_Population"],
#             "pred_relations": [],
#             "pred_concepts": [],
#         },
#         "Category": "A",
#         "Answer": [
#             "\"570922\"^^<http://www.w3.org/2001/XMLSchema#integer>"
#         ]
#     }

In [9]:
def f1score(y_pred,y_tar):
  true_positives = 0

  if len(y_tar)==0:
    if len(y_pred)==0:
      return 1

    else: # y_tar = -, y_pred!= -
      return "PRED-NOT_TAR" # target = - and predicted != - 

  if len(y_pred)!=0:
    for i in y_tar:
      if i in y_pred:
        true_positives+=1

    precision = true_positives/len(y_pred) # how many retrieved elements are relevant?
    recall = true_positives/len(y_tar) # how many relevant elements are retrieved?

    if true_positives == 0: # no element in common
      return 0

    else:
      f1 = 2*precision*recall/(precision + recall)
    return f1

  else:
    return 0 # predicted = 0

In [10]:
def matching_answers(answer_pred, answer_tar):
  
    # check if answers are the same
  if answer_tar == answer_pred:
    answerf1 = 1
  #if answers are not the same:
  elif 'POLYGON' in answer_tar:
    answerf1 = 0
  else:
    answerf1 = f1score(answer_pred, answer_tar)

  return answerf1

In [11]:
def instances_identified(tar_instances, pred_instances):
  if len(tar_instances)==0:
    return 1

  else:
    identified_instances = []
    for i in tar_instances:
      if i in pred_instances:
        identified_instances.append(i)

    instance_identification_proportion = len(identified_instances)/len(tar_instances)

    return(instance_identification_proportion)

# Step 0: Read files

In [12]:
import json

with open('GeoQuestions-target.json') as f:
  target = json.load(f)

with open('GeoQuestions-predicted.json') as f:
  predicted = json.load(f)

Mounted at /content/gdrive


In [13]:
indices = predicted.keys()

# Ablation Study Table

In [14]:
from decimal import Decimal
import pandas as pd

gq_data = []

for i in indices: 

  tar_instances = target[i]["Annotation"]["tar_instances"]
  pred_instances = predicted[i]["Annotation"]["pred_instances"]
  pred_instances_disambiguated = predicted[i]["Annotation"]["pred_instances_disambiguated"]

  tar_relations = target[i]["Annotation"]["tar_relations"]
  pred_relations = predicted[i]["Annotation"]["pred_relations"]

  tar_concepts =  target[i]["Annotation"]["tar_concepts"]
  pred_concepts = predicted[i]["Annotation"]["pred_concepts"]

  tar_properties =  target[i]["Annotation"]["tar_properties"]
  pred_properties = predicted[i]["Annotation"]["pred_properties"]

  answer_tar = target[i]["Answer"]
  answer_pred = predicted[i]["Answer"]

  # Answer = data[str(i)]["Answer"]
  inst_prop = instances_identified(tar_instances, pred_instances)
  inst_dis_f1 = f1score(pred_instances_disambiguated,tar_instances )
  rel_f1 =  f1score(pred_relations,tar_relations)
  conc_f1 = f1score(pred_concepts,tar_concepts)
  prop_f1 = f1score(pred_properties,tar_properties)
  answer_f1 = matching_answers(answer_pred, answer_tar)

  gq_data.append([ i, Decimal(str(inst_prop)).normalize(), Decimal(str(inst_dis_f1)).normalize(),  Decimal(str(rel_f1)).normalize(), Decimal(str(conc_f1)).normalize(), Decimal(str(prop_f1)).normalize(), Decimal(str(answer_f1)).normalize()])


gq_df = pd.DataFrame(gq_data, columns=['Index', 'Instances','Instances Disambiguated','Relations','Concepts', 'Properties', 'Answers'])
blankIndex=[''] * len(gq_df)
gq_df.index=blankIndex

In [15]:
gq_df

,Index,Instances,Instances Disambiguated,Relations,Concepts,Properties,Answers
,1,1,0,1,1,1,0
,144,1,1,0,1,1,1
,278,1,1,1,1,1,1
,433,1,1,1,1,1,0.9473684210526316
,454,1,1,1,0.5,1,1
